In [2]:
import numpy as np
import tensorflow as tf
import scipy
from tensorflow import keras
from tensorflow.keras import layers

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from tensorflow.keras.applications.densenet import DenseNet169
from keras.applications.densenet import preprocess_input

import matplotlib.pyplot as plt
from scipy import stats

from tqdm.notebook import tqdm

## Extracting features from VGG16

In [3]:
img_size = 224
densenet = DenseNet169(weights='imagenet', include_top=True, pooling='max', input_shape = (img_size, img_size, 3))

58548224/58541896 [==============================] - 1s 0us/step


In [4]:
op = densenet.get_layer('avg_pool').output
ip = densenet.input

basemodel = keras.Model(
    inputs=ip,
    outputs=op,
)

basemodel.trainable=False

In [5]:
images = np.empty((50, img_size, img_size, 3))
for i in range(50):
  image = load_img("data/" + str(i+1) +".jpg", target_size=(img_size, img_size))
  image = img_to_array(image)

  for j in range(img_size):
    for k in range(img_size):
      g = (image[j,k,0] + image[j,k,1] + image[j,k,2])/3
      image[j,k,0] = g
      image[j,k,1] = g
      image[j,k,2] = g

  images[i,:,:,:] = image

images = preprocess_input(images)
features = basemodel.predict(images)

colors = np.empty((5, img_size, img_size, 3))
for i in range(5):
  image = load_img("data/C" + str(i) + ".png", target_size=(img_size, img_size))
  image = img_to_array(image)
  colors[i,:,:,:] = image

colors = preprocess_input(colors)
colorfeatures = basemodel.predict(colors)

features = features.reshape(50,-1)
colorfeatures = colorfeatures.reshape(5,-1)

data = np.genfromtxt('data/bandw.csv', skip_header = 6, delimiter=',')
humanpred = np.zeros((50,5))
for i in range(50):
  for j in range(56):
    color = int(data[j, i]);
    humanpred[i, color] = humanpred[i, color]+1

prob = humanpred/56

In [6]:
inputimages = np.zeros((250,features.shape[1]))
inputcolors = np.zeros((250,features.shape[1]))
outputprob = np.zeros((250))

for i in range(50):
  for j in range(5):
    inputimages[5*i+j,:] = features[i,:]

for i in range(5):
  for j in range(50):
    inputcolors[5*j+i] = colorfeatures[i, :]

outputprob = prob.reshape(250)

## Model

In [7]:
def get_model(op_features=75):
  imageinputs = keras.Input(shape=(features.shape[1]), name="imageinputs")
  colorinputs = keras.Input(shape=(features.shape[1]), name="colorinputs")
  dense = layers.Dense(op_features, name = "linearlayer", activation='relu')
  imageresults = dense(imageinputs)
  colorresults = dense(colorinputs)
  result = tf.keras.layers.Dot(axes=1, normalize=True, name="dot")([imageresults, colorresults])

  model = keras.Model(
      inputs=[imageinputs, colorinputs],
      outputs=[result],
  )
  model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(lr=0.001), metrics = ['mean_squared_error'])
  
  return model

def get_corr(output, outputprob, vs, ve):
  x = np.copy(output.reshape(50,5))

  for i in range(50):
    rowsum = np.sum(x[i,:])
    for j in range(5):
      x[i,j] = x[i,j]/rowsum

  y = np.copy(outputprob.reshape(50,5))
  testx = x.reshape(-1)[vs:ve]
  testy = y.reshape(-1)[vs:ve]

  return scipy.stats.pearsonr(testx, testy)[0], scipy.stats.pearsonr(testx, testy)[1], testx

In [8]:
Rp = []
pvalue = []

for i in tqdm(range(50)):
  corr = [] #This contains correlation corresponding to specific validation data
  pred = np.zeros(250) #This will contain the final prediction for each of the loop. Total 5 subset of validation 50 each.

  # 5-fold cross validation 
  for vs in [0, 50, 100, 150, 200]:
    ve = vs + 50
    model = get_model(op_features=75)
    history = model.fit(
        {"imageinputs": inputimages[[*range(vs)] + [*range(ve, 250)]], "colorinputs": inputcolors[[*range(vs)] + [*range(ve, 250)]]},
        {"dot": outputprob[[*range(vs)] + [*range(ve, 250)]]},
        epochs=30,
        batch_size=10,
        shuffle=True,
        verbose=0,
    )
    output = model.predict({"imageinputs": inputimages, "colorinputs": inputcolors})
    R, _, pred[vs:ve] = get_corr(output, outputprob, vs, ve)
    corr.append(R)

  temp = get_corr(pred, outputprob, 0, 250)
  Rp.append(temp[0])
  pvalue.append(temp[1])

In [14]:
print(np.mean(Rp))
print(np.std(Rp))

0.5688877301533616
0.025721300786739548


In [10]:
Rp

[0.5678871220648656,
 0.5843625052287875,
 0.5367013274583408,
 0.5629719983239322,
 0.5844739128309859,
 0.6183085982237303,
 0.5275230673686596,
 0.5502323892742733,
 0.5773156961252721,
 0.5782484920608505,
 0.551967644341667,
 0.5930547641638512,
 0.5625405218212088,
 0.6131187546209287,
 0.5552739545029715,
 0.5724250018172106,
 0.5406556731601762,
 0.589546484299729,
 0.5316129337777218,
 0.5717196817136381,
 0.569209090216469,
 0.5704746065623734,
 0.6000046714942338,
 0.54791838542263,
 0.5896581791597094,
 0.6027751936413038,
 0.5479520616546383,
 0.5680395504930701,
 0.5670616474084492,
 0.5641481907564933,
 0.5205201251751295,
 0.6126495470695481,
 0.5686230257722199,
 0.5788761997022143,
 0.5736071449679842,
 0.6181650422884976,
 0.5389961907043898,
 0.5899611384111466,
 0.5789417936393054,
 0.5567423398850271,
 0.5227984984677638,
 0.607933835831052,
 0.5463465284486566,
 0.6182718083609042,
 0.5747161254297046,
 0.5539016589351016,
 0.5365754026865334,
 0.5642998469327338

In [11]:
print(np.mean(pvalue))
print(np.std(pvalue))

4.2795910752022494e-20
1.5683184574838671e-19
